# Abgabe 5 Mark Nagengast Porro, Tim Schwabe

In [28]:
import torch
from torch import nn
import gymnasium as gym


ImportError: DLL load failed: Das angegebene Modul wurde nicht gefunden.

In [ ]:
env =gym.make('CartPole-v1')


In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

AttributeError: module 'torch' has no attribute 'cuda'